In [ ]:
import numpy as np

data = open('data/shakespeare.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
chars = sorted(chars)

char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

In [ ]:
def build_data(data, T_x):
    X = []
    Y = []
    
    for i in range(T_x, len(data)):
        X.append(data[i - T_x:i])
        Y.append(data[i])
        
    return X, Y

T_x = 40
X, Y = build_data(data, T_x)

In [ ]:
def vectorization(X, Y, char_to_ix, vocab_size, T_x):    
    x = np.zeros((len(X), T_x, vocab_size))
    y = np.zeros((len(Y), 1, vocab_size))
    
    for i in range(len(X)):
        for j in range(len(X[i])):
            character = X[i][j]
            ix_character = char_to_ix[character]
            x[i, j, ix_character] = 1
        y[i, 0, ix_character] = 1
    
    return x, y

def vectorize(string, char_to_ix):
    vocab_size = len(char_to_ix)
    T_x = len(string)
    x = np.zeros((vocab_size, 1, T_x))
    for i in range(T_x):
        idx = char_to_ix[string[i]]
        x[idx, :, i] = 1
    
    return x

X_train, y_train = vectorization(X, Y, char_to_ix, vocab_size, T_x)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.losses import CategoricalCrossentropy
from keras.optimizers import Adam

classifier = Sequential()
classifier.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
classifier.add(Dropout(0.2))
classifier.add(LSTM(units = 50, return_sequences = False))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = X_train.shape[2], activation = 'softmax'))

classifier.compile(optimizer = Adam(learning_rate = 0.005), loss = CategoricalCrossentropy())

BATCH_SIZE = 32

y_train = np.squeeze(y_train)

classifier.fit(X_train, y_train, epochs = 50, batch_size = BATCH_SIZE)